In [2]:
import tensorflow as tf
from ray.rllib.offline.json_reader import JsonReader
import numpy as np
import numpy_indexed as npi
import pandas as pd
#from true_state_viewer import TrueStateTreeGraphViz, display_tree_pairs
from tqdm import trange
import os

/home/adamprice/.local/lib/python3.8/site-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import (exp, inf, pi, sqrt, floor, sin, cos, around, int,


In [5]:
def node_action(action, node):
    vec = np.zeros(3)
    if action < 2: 
        return vec
    action -= 2
    if action % 13 == node:
        #Analyse #Remove #Resotre
        vec[int(action) // 13] = 1
    return vec

In [2]:
data_path = '/home/adamprice/u75a-Data-Efficient-Decisions/CybORG/CybORG/Notebooks/logs/PPO/no_decoy_200000'
input_reader = JsonReader(data_path)
num_episodes = int(200000/100)
num_data_points = num_episodes * 99
state_length = 91
num_actions = 41

nodes = np.zeros((num_data_points*13, 7))
actions = np.zeros((num_data_points*13, 6))
node_id = np.zeros((num_data_points*13, 13))
next_nodes = np.zeros((num_data_points*13, 7))
exploit = np.zeros((num_data_points*13, 13))
scan = np.zeros((num_data_points*13, 13))
privileged = np.zeros((num_data_points*13, 13))
user = np.zeros((num_data_points*13, 13))
unknown = np.zeros((num_data_points*13, 13))
no  = np.zeros((num_data_points*13, 13))

index = 0
for i in trange(int(num_episodes/2)):
    samples = input_reader.next()
    ts = 0
    for i in range(samples['dones'].shape[0]-1):
        if samples['dones'][i]:
            continue
            ts = 0
        for n in range(13):
            encoding = np.zeros(13)
            encoding[n] = 1
            node_id[index,:] = encoding
            nodes[index,:] = samples['obs'][i][int(n*7):int(n*7)+7]
            if ts > 0:
                actions[index,:] = np.concatenate([node_action(samples['actions'][i], n), node_action(samples['actions'][i-1], n)])
            else: 
                actions[index,:] = np.concatenate([node_action(samples['actions'][i], n), np.array([0,0,0])])
            next_nodes[index,:] = samples['obs'][i+1][int(n*7):int(n*7)+7]
            exploit[index,:] = samples['obs'][i][np.arange(0,91,step=7)]
            scan[index,:] = samples['obs'][i][np.arange(1,91,step=7)]
            privileged[index,:] = samples['obs'][i][np.arange(3,91,step=7)]
            user[index,:] = samples['obs'][i][np.arange(4,91,step=7)]
            unknown[index,:] = samples['obs'][i][np.arange(5,91,step=7)]
            no[index,:]  = samples['obs'][i][np.arange(6,91,step=7)]
            index += 1
            ts += 1

#os.mkdir(data_path + '/data')
np.save(data_path + '/data/nodes.npy', nodes)
np.save(data_path + '/data/actions.npy', actions)
np.save(data_path + '/data/node_id.npy', node_id)
np.save(data_path + '/data/next_nodes.npy', next_nodes)
np.save(data_path + '/data/exploit.npy', exploit)
np.save(data_path + '/data/scan.npy', scan)
np.save(data_path + '/data/privileged.npy', privileged)
np.save(data_path + '/data/user.npy', user)
np.save(data_path + '/data/unknown.npy', unknown)
np.save(data_path + '/data/no.npy', no)

NameError: name 'JsonReader' is not defined

In [22]:
data_path = '/home/adamprice/u75a-Data-Efficient-Decisions/CybORG/CybORG/Notebooks/logs/PPO/no_decoy_200000'
input_reader = JsonReader(data_path)
num_episodes = int(200000/100)
num_data_points = num_episodes * 99
state_length = 91
num_actions = 41
seq_len = 10

states = np.zeros((num_data_points, seq_len, state_length))
rewards = np.zeros(num_data_points)
next_states = np.zeros((num_data_points, state_length))
action_hist_oh = np.zeros((num_data_points, seq_len, num_actions))
nodes = np.zeros((num_data_points*13, seq_len, 7))
actions = np.zeros((num_data_points*13, seq_len, 3))
node_id = np.zeros((num_data_points*13, seq_len, 13))
next_nodes = np.zeros((num_data_points*13, 7))
exploit = np.zeros((num_data_points*13, seq_len, 13))
scan = np.zeros((num_data_points*13, seq_len, 13))
privileged = np.zeros((num_data_points*13, seq_len, 13))
user = np.zeros((num_data_points*13, seq_len, 13))
unknown = np.zeros((num_data_points*13, seq_len, 13))
no  = np.zeros((num_data_points*13, seq_len, 13))

s_index = 0
index = 0
for i in trange(int(num_episodes/2)):
    samples = input_reader.next()
    ts = 0
    for i in range(samples['dones'].shape[0]-1):
        if samples['dones'][i]:
            ts = 0
            continue
        steps = ts if ts < seq_len else seq_len-1  
        states[s_index,:steps+1,:] =  samples['obs'][i-steps:i+1,:]
        next_states[s_index,:] = samples['obs'][i+1,:]
        rewards[s_index] = samples['rewards'][i]
        #print(samples['actions'][i-steps:i+1])
        action_hist_oh[s_index,:steps+1,:] = np.eye(41)[samples['actions'][i-steps:i+1]]
        s_index += 1
        # for n in range(13):     
        #     encoding = np.zeros((steps, 13))
        #     encoding[:steps+1,n] = 1
        #     node_id[index,:steps+1,n] = 1
        #     nodes[index,:steps+1,:] = samples['obs'][i-steps:i+1,int(n*7):int(n*7)+7]
        #     actions[index,:steps+1,:] = np.array([node_action(samples['actions'][i], n) for i in range(i-steps,i+1)])
        #     next_nodes[index,:] = samples['obs'][i+1,int(n*7):int(n*7)+7]
        #     exploit[index,:steps+1,:] = samples['obs'][i-steps:i+1,np.arange(0,91,step=7)]
        #     scan[index,:steps+1,:] = samples['obs'][i-steps:i+1,np.arange(1,91,step=7)]
        #     privileged[index,:steps+1,:] = samples['obs'][i-steps:i+1,np.arange(3,91,step=7)]
        #     user[index,:steps+1,:] = samples['obs'][i-steps:i+1,np.arange(4,91,step=7)]
        #     unknown[index,:steps+1,:] = samples['obs'][i-steps:i+1,np.arange(5,91,step=7)]
        #     no[index,:steps+1,:]  = samples['obs'][i-steps:i+1,np.arange(6,91,step=7)]
        #     index += 1
        ts += 1
    
data_path = data_path + '/data_seqence_10'
#os.mkdir(data_path)
#np.save(data_path + '/nodes.npy', nodes)
#np.save(data_path + '/actions.npy', actions)
np.save(data_path + '/actions_oh.npy', action_hist_oh)
# np.save(data_path + '/node_id.npy', node_id)
# np.save(data_path + '/next_nodes.npy', next_nodes)
# np.save(data_path + '/exploit.npy', exploit)
# np.save(data_path + '/scan.npy', scan)
# np.save(data_path + '/privileged.npy', privileged)
# np.save(data_path + '/user.npy', user)
# np.save(data_path + '/unknown.npy', unknown)
# np.save(data_path + '/no.npy', no)
# np.save(data_path + '/states.npy', states)
# np.save(data_path + '/next_states.npy', next_states)
# np.save(data_path + '/rewards.npy', rewards)

2023-04-21 08:06:48,763	WARNING json_reader.py:239 -- Treating input directory as glob patterns: ['/home/adamprice/u75a-Data-Efficient-Decisions/CybORG/CybORG/Notebooks/logs/PPO/no_decoy_200000/*.json', '/home/adamprice/u75a-Data-Efficient-Decisions/CybORG/CybORG/Notebooks/logs/PPO/no_decoy_200000/*.zip']
100%|██████████| 1000/1000 [00:19<00:00, 52.43it/s]


In [24]:
data_path = '/home/adamprice/u75a-Data-Efficient-Decisions/CybORG/CybORG/Notebooks/logs/PPO/no_decoy_200000'  + '/data_seqence_10'
a = np.load(data_path+'/actions_oh.npy')
a[1]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [4]:
np.save(data_path + '/states.npy', states)
np.save(data_path + '/next_states.npy', next_states)
np.save(data_path + '/rewards.npy', rewards)

In [46]:
data_path = '/home/adamprice/u75a-Data-Efficient-Decisions/CybORG/CybORG/Notebooks/logs/PPO/no_decoy_200000'
input_reader = JsonReader(data_path)
num_episodes = int(200000/100)
num_data_points = num_episodes * 99
state_length = 91
num_actions = 41

true_actions = np.zeros((num_data_points, num_actions))

index = 0
for i in trange(int(num_episodes/2)):
    samples = input_reader.next()
    ts = 0
    for i in range(samples['dones'].shape[0]-1):
        if samples['dones'][i]:
            continue
            ts = 0
        true_actions[index,samples['actions'][i]] = 1
        index += 1

np.save(data_path + '/data/true_actions.npy', true_actions)

2023-04-04 21:23:05,828	WARNING json_reader.py:239 -- Treating input directory as glob patterns: ['/home/adamprice/u75a-Data-Efficient-Decisions/CybORG/CybORG/Notebooks/logs/PPO/no_decoy_200000/*.json', '/home/adamprice/u75a-Data-Efficient-Decisions/CybORG/CybORG/Notebooks/logs/PPO/no_decoy_200000/*.zip']
100%|██████████| 1000/1000 [00:13<00:00, 73.73it/s]


In [7]:
data_path = '/home/adamprice/u75a-Data-Efficient-Decisions/CybORG/CybORG/Notebooks/logs/APPO/no_decoy_800000'
input_reader = JsonReader(data_path)
num_episodes = int(800000/100)
num_data_points = num_episodes * 99
state_length = 91
num_actions = 41
seq_len = 20

states = np.zeros((num_data_points, seq_len, state_length))
rewards = np.zeros(num_data_points)
next_states = np.zeros((num_data_points, state_length))
action_hist_oh = np.zeros((num_data_points, seq_len, num_actions))

s_index = 0
index = 0
for i in trange(int(num_episodes/10)):
    samples = input_reader.next()
    ts = 0
    for i in range(samples['dones'].shape[0]-1):
        if samples['dones'][i]:
            ts = 0
            continue
        steps = ts if ts < seq_len else seq_len-1  
        states[s_index,:steps+1,:] =  samples['obs'][i-steps:i+1,:]
        next_states[s_index,:] = samples['obs'][i+1,:]
        rewards[s_index] = samples['rewards'][i]
        action_hist_oh[samples['actions'][i-steps:i+1]] = 1
        s_index += 1
        ts += 1
    
data_path = data_path + '/data_seqence_20'
os.mkdir(data_path)
np.save(data_path + '/actions_oh.npy', action_hist_oh)
np.save(data_path + '/states.npy', states)
np.save(data_path + '/next_states.npy', next_states)
np.save(data_path + '/rewards.npy', rewards)

2023-04-19 09:30:43,070	WARNING json_reader.py:239 -- Treating input directory as glob patterns: ['/home/adamprice/u75a-Data-Efficient-Decisions/CybORG/CybORG/Notebooks/logs/APPO/no_decoy_800000/*.json', '/home/adamprice/u75a-Data-Efficient-Decisions/CybORG/CybORG/Notebooks/logs/APPO/no_decoy_800000/*.zip']
100%|██████████| 800/800 [01:27<00:00,  9.13it/s]


In [2]:
import numpy as np
t = np.arange(20).reshape((5,4))
t

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19]])

In [4]:
np.flip(t,axis=0)

array([[16, 17, 18, 19],
       [12, 13, 14, 15],
       [ 8,  9, 10, 11],
       [ 4,  5,  6,  7],
       [ 0,  1,  2,  3]])